<div style="border: 1px solid black">
<b><center><font size="4">Mini Projecto 01</font></center></b>

</div>

**Notebook base on**: [Ricardo Campos](https://www.di.ubi.pt/~rcampos)<br>
**Notebook Developed by**: Rene Jerez<br>
**email:**  rene.jerez@ubi.pt<br>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span>
<a href="#Tarefa 1" data-toc-modified-id="Tarefa 1-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Tarefa 1</a></span><ul class="toc-item"><li><span>
<a href="#100 ficheiros" data-toc-modified-id="100 ficheiros-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>100 ficheiros</a></span></li><li><span>
<a href="#Guardar json" data-toc-modified-id="Guardar json-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Guardar json</a></span></li><li><span>
<a href="#Carregar json" data-toc-modified-id="Carregar json-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Carregar json</a></span></li><li><span>
<a href="#Imprimir 5 json" data-toc-modified-id="Imprimir 5 json-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Imprimir 5 json</a></span></li><li><span>
<a href="#Nuvem de Palavras" data-toc-modified-id="Nuvem de Palavras-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Nuvem de Palavras</a></span></li></ul></li>
<li><span>
<a href="#Tarefa 2" data-toc-modified-id="Tarefa 2-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Tarefa 2</a></span><ul class="toc-item"><li><span>
<a href="#Wikipedia imagens" data-toc-modified-id="Wikipedia imagens-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Wikipedia imagens</a></span></li></ul></li>
<li><span>
<a href="#Tarefa 3" data-toc-modified-id="Tarefa 3-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Tarefa 3</a></span><ul class="toc-item"><li><span>
<a href="#API Search" data-toc-modified-id="API Search-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>API Search</a></span></li><li><span>
<a href="#Guardar dinamicamente" data-toc-modified-id="Guardar dinamicamente-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Guardar dinamicamente</a></span></li><li><span>
<a href="#Imprimir 5 json" data-toc-modified-id="Imprimir 5 json-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Imprimir 5 json</a></span></li><li><span>
<a href="#Prêmio Arquivo PT" data-toc-modified-id="Prêmio Arquivo PT-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Prêmio Arquivo PT</a></span></li></ul></li>



</div>

# Tarefa 1

 Familiarização com a obtenção de dados a partir de ficheiros pdf

1. Reúna um conjunto aproximado de 100 ficheiros em formato pdf relacionados com
uma temática à sua escolha (e.g., artigos científicos; documentos do parlamento
europeu; patentes; programas eleitorais; etc). Proceda à extração do texto de cada
ficheiro com recurso a bibliotecas Python.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# The URL of the page containing the links
page_url = 'https://www.ubi.pt/Pessoas'

# Fetch the content of the page
response = requests.get(page_url)
page_content = response.content

# Parse the page content using BeautifulSoup
soup = BeautifulSoup(page_content, 'html.parser')

# Find all links in the page (assuming they are contained in <a> tags)
links = soup.find_all('a')

# Create a list to hold your links and texts
link_table = []

# Loop through all found <a> tags
for link in links:
    href = link.get('href')  # Extract the href attribute (the URL)

    # Skip fragment identifiers and ensure the URL is complete
    if href and not href.startswith('#'):
        # Optional: Concatenate with the base URL if href is a relative URL
        if not href.startswith('http'):
            href = requests.compat.urljoin(page_url, href)

        # Store the link and an empty string for the text in the table
        link_table.append([href, ""])

# Convert the list to a DataFrame for easier handling
df = pd.DataFrame(link_table, columns=['Link', 'Text'])

# Filter for links containing 'Pessoa'
df = df[df['Link'].str.contains('Pessoa', na=False)]

# Remove duplicate links
df.drop_duplicates(subset='Link', inplace=True)

# Display the DataFrame
# print(df)

# Ensure the DataFrame has the new columns 'ColunaNome' and 'Photo' with default values
df['ColunaNome'] = "Not found"
df['Photo'] = "Not found"

# Now iterate over the DataFrame, access each link, and fetch the text, ColunaNome content, and photo URL
for index, row in df.iterrows():
    link_url = row['Link']

    try:
        # Fetch the content of the linked page
        linked_page_response = requests.get(link_url)
        linked_page_content = linked_page_response.content

        # Parse the linked page
        linked_soup = BeautifulSoup(linked_page_content, 'html.parser')

        # Find the specific div by its id for 'Text' column
        specific_div = linked_soup.find('div', id="ConteudoHolder_LV_Detalhe_Pessoa_Panel_Visibilidade_0")
        # Extract text from the specific div, if it exists
        if specific_div:
            text = specific_div.get_text(separator=' ', strip=True)
        else:
            text = "Specific div not found."
        # Update the 'Text' column in the DataFrame with the extracted text
        df.at[index, 'Text'] = text

        # Find elements by class name for 'ColunaNome' column
        coluna_nome_elements = linked_soup.find_all(class_="colunaNome")
        # Extract text from these elements, if they exist
        if coluna_nome_elements:
            coluna_nome_texts = [elem.get_text(separator=' ', strip=True) for elem in coluna_nome_elements]
            # Join all found texts into a single string (or handle as you see fit)
            coluna_nome_text = ' | '.join(coluna_nome_texts)
        else:
            coluna_nome_text = "colunaNome elements not found."
        # Update the 'ColunaNome' column in the DataFrame with the extracted text
        df.at[index, 'ColunaNome'] = coluna_nome_text

        # Find the first element with class 'blkFicheiro ficheiroImagem' and extract the 'src' of the img tag
        photo_element = linked_soup.find(class_="blkFicheiro ficheiroImagem")
        if photo_element and photo_element.find('img'):
            photo_src = photo_element.find('img')['src']
            # Optional: Concatenate with the base URL if 'src' is a relative URL
            if not photo_src.startswith('http'):
                photo_src = requests.compat.urljoin(link_url, photo_src)
        else:
            photo_src = "Photo not found."
        # Update the 'Photo' column in the DataFrame with the photo URL
        df.at[index, 'Photo'] = photo_src

    except requests.exceptions.RequestException as e:
        # Handle exceptions like missing schema, connection errors, etc.
        print(f"Error fetching {link_url}: {e}")
        df.at[index, 'Text'] = "Error fetching content."
        df.at[index, 'ColunaNome'] = "Error fetching content."
        df.at[index, 'Photo'] = "Error fetching content."

    # Be mindful of the website's policy on automated requests and add a delay if necessary


In [28]:
df.head(5)

Link  \
33                  https://www.ubi.pt/Pessoas   
225      https://www.ubi.pt/Pessoa/abel.abejas   
227         https://www.ubi.pt/Pessoa/ajpgomes   
229           https://www.ubi.pt/Pessoa/abilio   
231  https://www.ubi.pt/Pessoa/adelaide.santos   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

Desde o seu surgimento que o JSON rapidamente se tornou no "de facto standard" para troca de informação. A obtenção de um ficheiro JSON decorre habitualmente no decurso da execução de uma API. A figura abaixo ilustra um ficheiro JSON:

<img src="https://www.di.ubi.pt/~rcampos/assets/img_tutorials/Text2Analytics/json.jpg" width="500" height="500" align="center"/>

Como pode ver, o JSON suporta tipos de dados primitivos (strings, numbers, floats, etc), assim como listas encadeadas e objetos.

O código seguinte permite obter, em formato JSON, os resultados da api [textsearch](https://github.com/arquivo/pwa-technologies/wiki/Arquivo.pt-API) do [arquivo.pt](https://arquivo.pt/).

No código, fazemos uso de headers. API headers are like an extra source of information for each API call you make. Their job is to represent the meta-data associated with an API request and response. If you ever encounter issues with an API, the first place you should look is the headers, since they can help you track down any potential issues. [https://serpapi.com/blog/how-to-reduce-chance-of-being-blocked-while-web/]

A user agent (any software, acting on behalf of a user) is a string that a browser or application sends to each website you visit. A typical user agent string contains details like – the application type, operating system, software vendor, or software version of the requesting software user agent. Web servers use this data to assess the capabilities of your computer, optimizing a page’s performance and display. **Most websites block requests that come in without a valid browser as a User-Agent**. User-Agents are sent as a request header called `User-Agent`.

This is the most basic header and usually, for most websites, it will be enough, but user-agent does not guarantee that your request won't be declined or blocked. In basic explanation, user-agent is needed to act as a "real" user visit, which is also known as user-agent spoofing, when bot or browser send a fake user-agent string to announce themselves as a different client.

**The reason why request might be blocked is that, for example in Python requests library, default user-agent is python-requests and websites understands that it's a bot and might block a request in order to protect the website from overload, if there's a lot of requests being sent.**

Para saber o seu user-agent consulte o seguinte link: https://router-network.com/tools/what-is-my-user-agent

Adicionalmente, é possível especificar um [timeout](https://linuxpip.org/python-requests-timeout/): 

- The **connect** timeout is the number of seconds Requests will wait for your client to establish a connection to a remote machine

- Once your client has connected to the server and sent the HTTP request, the **read** timeout is the number of seconds the client will wait for the server to send a response. (Specifically, it’s the number of seconds that the client will wait between bytes sent from the server. In 99.9% of cases, this is the time before the server sends the first byte).

Most requests to external servers should have a timeout attached, in case the server is not responding in a timely manner. By default, requests do not time out unless a timeout value is set explicitly. Without a timeout, your code may hang for minutes or more.

O exemplo abaixo especifica um timeout de **5seg** (tanto para o **connect** como para o **read**). Se o cliente não conseguir estabelecer conexão com o servidor no espaço desses 5seg, ou, se após a conexão o servidor da API não devolver a resposta no espaço de 5seg, a biblioteca requests lança uma exceção e termina a conexão com o servidor. Note que terminar uma conexão não é sinómino de que o processo termine no servidor. Na verdade o servidor vai continuar a executar o pedido do utilizador (uma vez que este já está em execução), simplesmente não o vai devolver ao utilizador (o que é particularmente útil no caso de o sistema efetuar cache dos pedidos do utilizador, significando que a resposta estará em cache aquando do próximo pedido).

Se pretender especificar timeouts diferentes para o **connect** e para o **read** deverá fazê-lo desta forma: `timeout=(5, 27)`.

In [ ]:
import requests
r = requests.get(https://arquivo.pt/textsearch, params={'q': "António Costa"})

In [ ]:
import requests
from requests.exceptions import Timeout

url_api = "https://arquivo.pt/textsearch"

query = "António Costa"

headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0 "}

payload = {'q': query}
try:
    r = requests.get(url_api, params=payload, headers=headers, timeout = 50)
    print(r.url)
except Timeout:
    print('Timeout has been raised.')

Para visualizar os resultados pode pegar no URL gerado e colocá-lo num browser (e.g., Firefox). Note que a visualização dos resultados no Chrome obriga à instalação de um extensão (e.g., JSon View: https://chrome.google.com/webstore/detail/jsonview/chklaanhfefbnpoihckbnefhakgolnmc?hl=pt-PT].

O método `json` do módulo `requests` converte esse texto numa estrutura de Python (e.g. ,dicionário) permitindo dessa forma que o JSON possa ser usado nos programas.

In [ ]:
type(r.json())

O fato de ser um dicionário permite-nos imprimir todo o conteúdo do ficheiro JSON. Por uma questão de legibilidade vamos atribuir o json obtido a uma variável denominada `content` e que passaremos a usar daqui para frente:

In [ ]:
content = r.json()
content

In [ ]:
#Impressao com recurso ao pretty print - mais entendivel
import pprint
pprint.pprint(content['response_items'])

Ou ainda mais entendível:

In [ ]:
import json
print(json.dumps(content['response_items'], indent=4))

Para iterar sobre os resultados recorremos a um ciclo for:

In [ ]:
for element in content['response_items']:
    print(element['title'])
    print(element['linkToArchive'])
    print("\n")

Recorra à api do Arquivo.pt (`https://arquivo.pt/textsearch`) para obter notícias acerca do `barack obama`. Restrinja a sua pesquisa ao site do jornal público (considere o seguinte parâmetro: 'siteSearch': 'http://www.publico.pt') e a um total de 2000 maxitems (considere o seguinte parâmetro: 'maxItems':2000).

In [ ]:
import requests
from requests.exceptions import Timeout

#especifique o endpoint
url_api = ""

#especifique a sua query
query = ""

headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0 "}

#acrescente ao payload o parâmetro sitesearch e o parâmetro maxItems
payload = {'q': query}

try:
    r = requests.get(url_api, params=payload, headers=headers, timeout = 5)
    print(r.url)
except Timeout:
    print('Timeout has been raised.')

Clique duas vezes <b>aqui</b> para aceder à solução completa
<!--
import requests
from requests.exceptions import Timeout

#especifique o endpoint
url_api = "https://arquivo.pt/textsearch"

#especifique a sua query
query = "barack obama"

headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0 "}

#acrescente ao payload o parâmetro sitesearch  e o parâmetro maxItems
payload = {'q': query, 'siteSearch': 'http://www.publico.pt', 'maxItems':2000}}

try:
    r = requests.get(url_api, params=payload, headers=headers, timeout = 5)
    print(r.url)
except Timeout:
    print('Timeout has been raised.')
-->

Imprima, para cada elemento obtido, o `title`, o `originalURL` e o `tstamp`:

In [ ]:
content = r.json()

for element in content['response_items']:
    #imprima o title
    #imprima o originalURL
    #imprima o tstamp
    print("\n")

Clique duas vezes <b>aqui</b> para aceder à solução completa
<!--
content = r.json()

for element in content['response_items']:
    print(element['title'])
    print(element['originalURL'])
    print(element['tstamp'])
    print("\n")
-->

Escreva um programa que guarde o conteúdo da variável `content` num ficheiro json denominado `arquivopt.json`.

In [ ]:
#escreva aqui o seu programa

Clique duas vezes <b>aqui</b> para aceder à solução completa
<!--
import json

path = "data/"

file_name = "arquivopt.json"

#gravar ficheiro com recurso a dump
with open(path + file_name, "w") as outfile:
    json.dump(content, outfile)
-->

<hr>

### APIs

Neste módulo vamos explicar a forma de consumir web services no Python através do uso de APIs. Em concreto vamos consumir resultados a partir do:
- New York Times
- NIF.PT
- Arquivo.pt images search

#### New York Times

No exercício abaixo vamos obter dados do New York Times. Para tal, deverá começar por criar uma conta, registar uma app, para finalmente pode aceder à sua API key. Veja como [aqui](https://developer.nytimes.com/get-started)

Atualmente o NYT oferece acesso gratuito a uma série de [APIs](https://developer.nytimes.com/apis):

- [Archive API](https://developer.nytimes.com/docs/archive-product/1/overview): The New York Times Archive API allows developers to retrieve past New York Times articles by month going back to 1851. Users can get all articles for a given month as a JSON object by passing the desired year and month (both are required parameters). The Archive API is very useful if you want to build your own database of NYT article metadata. The response size can be large (~20mb).
- [Article Search API](https://developer.nytimes.com/docs/articlesearch-product/1/overview): Use the Article Search API to look up articles by keyword.
- [Books API](https://developer.nytimes.com/docs/books-product/1/overview): The Books API provides information about book reviews and The New York Times Best Sellers lists.
- [Community API](https://developer.nytimes.com/docs/community-api-product/1/overview): Get comments from registered users on New York Times articles.
- [Geographic API](https://developer.nytimes.com/docs/geo-product/1/overview): The New York Times Geographic API lets developers to get access to locations of the people, places, organizations, and descriptors that make up the controlled vocabulary found in the New York Times's metadata. 
- [Most Popular API](https://developer.nytimes.com/docs/most-popular-product/1/overview): The New York Times Most Popular API provides links and information about the most frequently e-mailed, shared and viewed blog posts and articles on the New York Times website. Developers can look at popularity based on views, emails and shares. In the case of the last method, developers can also look at shares on specific social networks, including Twitter, Facebook, Digg and more.
- [Movie Reviews API](https://developer.nytimes.com/docs/movie-reviews-api/1/overview): The New York Times Movie Review API gives users access to movie reviews by the New York Times critics. The API gives access to over 22,000 New York Times movie reviews from today back to 1924. Using the Movie Reviews API you can query reviews and get data such as byline, critics pick, date updated, mpaa rating, title, headline, opening date, article link and multimedia. Users can search movie reviews by keywords or critic.
- [Semantic API](https://developer.nytimes.com/docs/semantic-api-product/1/overview): The Semantic API complements the Articles API. With the Semantic API, you get access to the long list of people, places, organizations and other locations, entities and descriptors that make up the controlled vocabulary used as metadata by The New York Times (sometimes referred to as Times Tags and used for Times Topics pages.
- [TimesTags API](https://developer.nytimes.com/docs/timestags-product/1/overview): New York Times has its own set of "controlled meta-data," which it uses to categorize its stories. The TimesTags API provides access to this meta-data. 
- [Times Newswire API](https://developer.nytimes.com/docs/timeswire-product/1/overview): With the Times Newswire API, you can get links and metadata for Times articles as soon as they are published on NYTimes.com. The Times Newswire API provides an up-to-the-minute stream of published items.
- [Top Stories](https://developer.nytimes.com/docs/top-stories-product/1/overview): The New York Times Top Stories API returns an array of articles currently on the specified section for arts, business and more. This includes services to access data for; arts, automobiles, books, business, fashion, food and more. 

No código abaixo vamos fazer uso da `Article Search API`

**Article Search API**

The NYT Article Search API allows you to search more than 2.8 million New York Times articles from `1981 to today`, retrieving headlines, abstracts, lead paragraphs and links to associated multimedia.

The API supports the following type of searching: 
- Standard keyword searching
- Date range: all articles from X date to Y date
- Field search: search within any number of given fields, e.g., title:obama byline:dowd
- Conjunction and disjunction (AND and NOT) operations, e.g., baseball yankees -"red sox"
- Ordering by closest (variable ranking algorithms), newest and oldest

The Article Search API also offers faceted searching. The available facets include Times-specific fields such as sections, taxonomic classifiers and controlled vocabulary terms (names of people, organizations and geographic locations).

In [ ]:
import json

file_name = "keys.json"

with open(file_name, "r") as readfile:
    data = json.load(readfile)

key = data['nyt']

In [ ]:
import requests
URL_REQUEST = "https://api.nytimes.com/svc/search/v2/articlesearch.json"

No exemplo abaixo pretendemos obter todos os artigos relacionados com a query `Donald Trump`.

In [ ]:
query = "Donald Trump"
sort = "relevance"

payload = {
    'q': query, 
    'api-key': key,
    'sort': sort
}

headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0 "}

r = requests.get(URL_REQUEST, params=payload, headers=headers)
print(r.url)

In [ ]:
contentsJSon = r.json()
contentsJSon

O código seguinte permite imprimir os resultados obtidos.

In [ ]:
import datetime

print(f"total = {contentsJSon['response']['meta']['hits']}\n")
for doc in contentsJSon['response']['docs']:
    print(f"Source = {doc['source']}")
    print(f"URL = {doc['web_url']}")
    print(f"News Desk = {doc['news_desk']}")
    print(f"PubDate = {datetime.datetime.strptime(doc['pub_date'], '%Y-%m-%dT%H:%M:%S+0000').strftime('%d-%m-%Y')}")
    print(f"Snippet = {doc['snippet']}")
    print(f"Abstract = {doc['abstract']}")
    print(f"MainHeadline = {doc['headline']['main']}")
    print(f"PrintHeadline = {doc['headline']['print_headline']}")
    print(f"LeadParagraph = {doc['lead_paragraph']} \n")

De seguida vamos guardar os resultados num conjunto de listas.

In [ ]:
import datetime

ListOfResults = []

for doc in contentsJSon['response']['docs']:
    ListOfResults.append({'source': doc['source'], 
                          'web_url': doc['web_url'],
                          'news_desk': doc['news_desk'],
                          'date': datetime.datetime.strptime(doc['pub_date'], "%Y-%m-%dT%H:%M:%S+0000").strftime("%d-%m-%Y"),
                          'snippet': doc['snippet'],
                          'abstract': doc['abstract'],
                          'headline_main': doc['headline']['main'],
                          'headline_print': doc['headline']['print_headline'],
                          'lead_paragraph': doc['lead_paragraph']
                         })

ListOfResults

Na posse destes dados podemos agora mostrá-los num dataframe pandas.

In [ ]:
import pandas as pd

df = pd.DataFrame(ListOfResults)
df

Para guardar o dataframe em HTML use o seguinte código.

In [ ]:
df.to_html("donald trump.html")

ou:

In [ ]:
html = df.to_html()
with open("donald trump.html", "w", encoding="utf-8") as file:
    file.write(html)

Poderiamos agora por exemplo restringir os resultados ao news_desk Magazine.

In [ ]:
df[df['news_desk'] == 'Politics']

#### NIF.PT

O [NIF.PT](https://www.nif.pt) é um serviço gratuito de pesquisa de NIF portugueses, que permite procurar, descobrir e consultar NIF de empresas. Para facilitar a tarefa de validar programaticamente um nif, este website disponibiliza uma api.

Recorra à [api](https://www.nif.pt/api) disponibilizada pelo NIF.PT para consultar informações acerca do número de contribuinte de uma empresa à sua escolha. Imprima o resultado do campo `title`, `address`, `pc4-pc3`, `city`, `activity`, `cae`, `email`, `phone`, `website`, `fax`.

In [ ]:
import json

file_name = "keys.json"

with open(file_name, "r") as readfile:
    data = json.load(readfile)

key = data['nif']

In [ ]:
#Insira o seu código aqui

import requests
from requests.exceptions import Timeout

url_api = "http://www.nif.pt/?json=1"

nif = "501836926"

headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0 "}

payload = {'q': nif, 'key':key}
try:
    r = requests.get(url_api, params=payload, headers=headers, timeout = 50)
    print(r.url)
except Timeout:
    print('Timeout has been raised.')

In [ ]:
content = r.json()
content

Clique duas vezes <b>aqui</b> para aceder à solução
<!--
import requests
#Nao se esqueca de inserir a key no codigo abaixo
key = ''
NIF = 501836926
url_api = 'http://www.nif.pt/?json=1'
payload = { 'q': NIF, 'key': key}

headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0 "}

r = requests.get(url_api, params=payload, headers=headers)
print(r.url)
contents =r.json()
print(contents)
-->

Clique duas vezes <b>aqui</b> para aceder ao resto da solução
<!--
content = contents['records'][str(NIF)]
print(f"Title = {content['title']}")
print(f"Address = {content['address']}")
print(f"Postal Code = {content['pc4']} - {content['pc3']}")
print(f"City = {content['city']}")
print(f"Activity = {content['activity']}")
print(f"CAE = {content['cae']}")
print(f"Email = {content['contacts']['email']}")
print(f"Phone = {content['contacts']['phone']}")
print(f"Website = {content['contacts']['website']}")
print(f"Fax = {content['contacts']['fax']}")
-->

Desenhe agora um programa que lhe permita obter a informação (title e address) de dois NIFs à sua escolha. Não se esqueça que o serviço NIF.pt apenas permite um único request por minuto.

In [ ]:
#Insira o seu código aqui

Clique duas vezes <b>aqui</b> para aceder à solução
<!--
import requests
import time

import json

file_name = "keys.json"

with open(file_name, "r") as readfile:
    data = json.load(readfile)

key = data['nif']

url_api = 'http://www.nif.pt/?json=1'

ListOfNIFs = [501836926, 510462553]

ListOfResults = []

headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0 "}

for nif in ListOfNIFs:
    payload = { 'q': nif, 'key': key}
    
    r = requests.get(url_api, params=payload, headers = headers)
    contents =r.json()
    
    title = contents['records'][str(nif)]['title']
    address = contents['records'][str(nif)]['address']
    
    ListOfResults.append({'title':title, 'address': address})
    time.sleep(61)
    
ListOfResults
-->

Finalmente envie a informação coletada para um dataframe.

In [ ]:
#Insira o seu código aqui

Clique duas vezes <b>aqui</b> para aceder à solução
<!--
import pandas as pd
df = pd.DataFrame(ListOfResults)
df
-->

#### Arquivo.pt Images Search

Considere a seguinte página web (https://github.com/arquivo/pwa-technologies/wiki/ImageSearch-API-v1.1-(beta) que descreve o funcionamento da API de imagens do Arquivo.pt. Escreva um programa que permita obter dados a partir dessa API.

Clique duas vezes <b>aqui</b> para aceder à solução
<!--
import requests
URL_REQUEST = "https://arquivo.pt/imagesearch"
query = 'Avatar'
payload = {
    'q': query
}

headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0 "}

r = requests.get(URL_REQUEST, params=payload, headers = headers)
print(r.url)
contents = r.json()
for content in contents['responseItems']:
    print(content['imgLinkToArchive'])
-->

#### Wikipedia

O código seguinte exemplifica a obtenção de texto a partir da API da wikipedia. Os resultados aqui devolvidos são iguais aos obtidos a partir do package python da wikipedia.

In [ ]:
import requests
from requests.exceptions import Timeout

url_api = "https://pt.wikipedia.org/w/api.php"

query = "António Costa"

headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0 "}

payload = {'action': 'query',
            'format': 'json',
            'titles': query,
            'prop': 'extracts',
            'exintro': True,
            'explaintext': True
        }

try:
    r = requests.get(url_api, params=payload, headers=headers, timeout = 5)
    print(r.url)
except Timeout:
    print('Timeout has been raised.')

Da execução do código seuinte é possível observar que o texto relativo a cada entidade aparece associado a uma página cujo número é dinamicamente obtido aquando da chamada à API (98576 no caso da query ser António Costa; 1286 no caso de ser Mário Soares) dificultando dessa forma o acesso aos dados.

In [ ]:
content = r.json()
content

De facto, a obtenção dos dados em "extract" obrigaria à especificação do número em questão, algo impraticável de automatizar:

In [ ]:
content['query']['pages']['98576']['extract']

De forma a ultrapassarmos essa dificuldade recorremos ao comando `next-iter` que nos permite aceder programaticamente a cada uma das chaves do dicionário. Assim, `next(iter(content['query']))` dá-nos acesso à próxima chave, pages:

In [ ]:
next(iter(content['query']))

Da mesma forma que `next(iter(content['query']['pages']))` nos dá acesso à chave `98576`:

In [ ]:
next(iter(content['query']['pages']))

Chegados aqui podemos recorrer a `values` para obter os valores associados à chave `98576`.

In [ ]:
next(iter(content['query']['pages'].values()))

E em concreto ao texto existente no campo `extract`:

In [ ]:
next(iter(content['query']['pages'].values()))['extract']

<hr>

### Other APIs

- [Public API Portugal](https://github.com/devpt-org/public-data-portugal)
- [Awesome Portugal Data](https://github.com/rgllm/awesome-portugal-data)
- [Covid19](https://covid19api.com/)
- [News API](https://newscatcherapi.com/)
- [Media Cloud](https://mediacloud.org/support)
- [Even Registry](https://eventregistry.org/)
- [Google Search API](https://serpapi.com/)
- [Dandelion API](https://dandelion.eu/)
- [The best api for live music](https://www.songkick.com/developer)
- [lexalytics](https://www.lexalytics.com/)
- [9 crime APIs](https://rapidapi.com/collection/crime)
- [top-50 most popular APIs 2020](https://rapidapi.com/blog/most-popular-api/?utm_source=google&utm_medium=cpc&utm_campaign=Beta_100613405446&utm_term=%2Bpopular%20%2Bapi_b&gclid=Cj0KCQjwoJX8BRCZARIsAEWBFMK-3YAO7LENBEwh2P7rE82Go5jgCST8xxwC36fN7v3DUtGwHw2mz90aAosIEALw_wcB)